# Naive Bayes Algorithm

*Intro to Statistical Learning*: Book Sections: 4.4.4 Naive Bayes

*Ethical Algorithm*: Chapter 1, Algorithmic Privacy, A Bad Solution, (Page 26-31)

The example on github uses the Cleveland Heart Disease data set from UCI
https://archive.ics.uci.edu/ml/datasets/Heart+Disease


**14 attributes used:**

1. (age)  
2. (sex)  (1 = male, 0 = female)
3. (cp) = presence of chest pain (1 = typical angina, 2 = atypical angina, 3 = non-anginal pain, 4 = asymptomatic)
4. (trestbps) = resting blood pressure in mmHg
5. (chol) = cholesterol level in mg/dL
6. (fbs) = fasting blood sugar > 120 mg/dl (1 = yes , 0 = no)
7. (restecg)  = resting electrocardiographic level (0 = normal, 1 = having ST-T wave abnormality, 2 = showing probable or definite left ventricular hypertrophy)
8. (thalach)  = maximum heart rate achieved 
9. (exang)  = exercise induced angina (1 = yes; 0 = no) 
10. (oldpeak)  = ST depression induced by exercise relative to rest 
11. (slope) = the slope of the peak exercise ST segment (1 = upsloping, 2 = flat, 3 = downsloping)
12. (ca)  = number of major vessels colored by fluoroscopy  (0-3)
13. (thal)  = (3 = normal; 6 = fixed defect; 7 = reversible defect)
14. (num) = diagnosis of heart disease (this is the class that we want to predict; 0 = no heart disease, 1 = heart disease)

**Classes:**
0 = the patient has no presence of heart disease
1= the patient has heart disease


## Section 1: The Essence of Data
**What is Naive Bayes?**

Naive Bayes is a probabilistic algorithm that can be used for classification or clustering. The model predicts the instance’s class based on the instance's attributes using conditional probabilities.

Example: Spam filters in email applications use Naive Bayes to decide if an email is spam or not.

There are three types of Naive Bayes classification algorithms:
1. Gaussian Naive Bayes should be used for continuous values. It is assumed that the probability of these values follows a normal distribution.
2. Multinomial Naive Bayes should be used for discrete values.
3. Bernoulli Naive Bayes should be used if the value is a random variable that can only be one of two values (such as 0 and 1).

**How does Naive Bayes work?**

The algorithm predicts the class that the instance belongs to using Baye’s Theorem for conditional probabilities.

<div align = 'center'><font size = '24'>$P(A|B)=\frac{P(B|A)P(A)}{P(B)}$</font></div>

Example: from the heart disease dataset

<div align = 'center'><font size = '4'>$P(heart\ disease|male)=\frac{P(male|heart\ disease)P(heart\ disease)}{P(male)}=\frac{\frac{114}{139}\times\frac{139}{303}}{\frac{206}{303}}=0.5534$</font></div>

**Two types of problems:**

**Classification**- The input instances (from the training data) are classified based on their class labels (whatever parameter we are trying to predict).  It is used for supervised learning because training data is required. Naive Bayes is typically used for classification.
The heart disease tutorial uses classification methods.

**Clustering-** There are no class labels and the instances are grouped together based on their similarities. Clustering is used for unsupervised learning.

**What assumptions are made?**

It is called “Naive” because it is assumed that all attributes are independent of each other.

**What are the issues that these assumptions can cause?**

All of the attributes are not necessarily independent of each other.
For example in the Cleveland Heart Disease dataset exercise induced angina can be correlated to the presence of stationary angina. Also, resting blood pressure and cholesterol levels are correlated. Bayesian Network (Bayes net) is a probabilistic graphical model that extends on capabilities of Naive Bayes by modeling specific dependencies between the attributes. In the most general form this is represented by a directed graph where each edge represents a conditional probability distribution between two attribute nodes. 

Related Articles:
Harvard Business Review: 10 Promising AI Applications in Health Care:

https://hbr.org/2018/05/10-promising-ai-applications-in-health-care?ab=at_art_art_1x4_s01 

**Related Concepts- Bayes decision boundary, test error
Bayes Decision Boundary**

![](./fig/Naive/fig1.png)

The purple dashed line represents the Bayes decision boundary. The decision boundary represents the points where the probability is exactly 50% of belonging to either class.
The orange grid indicates the region in which a test observation will be assigned to the orange class, and the blue grid indicates the region in which a test observation will be assigned to the blue class.


data bias is reflected as algorithmic error


# Example Codes

In [2]:
import numpy as np
import pandas as pd
import math
import sklearn
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [18]:
#load file as a dataframe
#df = pd.read_csv('processed.cleveland.csv')
dataset = pd.read_csv('processed.cleveland.csv')
#df
dataset

,age,sex,chestpain,restbps,cholesterol,fastingbs,restecg,thalach,exerciseang,oldpeak,slope,ca,thal,heartdisease
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0,7,1
299,68,1,4,144,193,1,0,141,0,3.4,2,2,7,1
300,57,1,4,130,131,0,0,115,1,1.2,2,1,7,1
301,57,0,2,130,236,0,2,174,0,0.0,2,1,3,1


In [76]:
#check descriptive statistics of dataset
df.describe()

,age,sex,chestpain,restbps,cholesterol,fastingbs,restecg,thalach,exerciseang,oldpeak,slope,ca,thal,heartdisease
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.438944,0.679868,3.158416,131.689769,246.693069,0.148515,0.990099,149.607261,0.326733,1.039604,1.600660,0.679868,4.722772,0.458746
std,9.038662,0.467299,0.960126,17.599748,51.776918,0.356198,0.994971,22.875003,0.469794,1.161075,0.616226,0.934878,1.938383,0.499120
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
25%,48.000000,0.000000,3.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
50%,56.000000,1.000000,3.000000,130.000000,241.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000,3.000000,0.000000
75%,61.000000,1.000000,4.000000,140.000000,275.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000,7.000000,1.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,7.000000,1.000000


In [77]:
#checking for null values
df.isna().sum()

age             0
sex             0
chestpain       0
restbps         0
cholesterol     0
fastingbs       0
restecg         0
thalach         0
exerciseang     0
oldpeak         0
slope           0
ca              0
thal            0
heartdisease    0
dtype: int64

In [78]:
#separate target attribute
target = dataset.heartdisease
predictors = dataset.drop('heartdisease', axis = 'columns')

In [79]:
#split data into 30% test and 70% training
x_train, x_test, y_train, y_test =  train_test_split(predictors,target,test_size = 0.3)

In [80]:
#number of instances in train set

len(x_train)

212

In [81]:
#number of instances in test set

len(x_test)

91

In [82]:
#train the model using the Gaussian model probabilities
model = GaussianNB()
model.fit(x_train,y_train)

GaussianNB()

In [83]:
#first 10 instances from the test set
x_test[:10]

,age,sex,chestpain,restbps,cholesterol,fastingbs,restecg,thalach,exerciseang,oldpeak,slope,ca,thal
238,49,0,2,134,271,0,0,162,0,0.0,2,0,3
284,61,1,4,148,203,0,0,161,0,0.0,1,1,7
279,58,0,4,130,197,0,0,131,0,0.6,2,0,3
61,46,0,3,142,177,0,2,160,1,1.4,3,0,3
130,54,1,3,120,258,0,2,147,0,0.4,2,0,7
300,57,1,4,130,131,0,0,115,1,1.2,2,1,7
70,65,0,3,155,269,0,0,148,0,0.8,1,0,3
49,53,1,3,130,197,1,2,152,0,1.2,3,0,3
54,60,1,4,130,253,0,0,144,1,1.4,1,1,7
250,57,1,4,110,201,0,0,126,1,1.5,2,0,6


In [84]:
# actual instances of heart disease for the first 10 instances from the test set
y_test[:10]

238    0
284    1
279    0
61     0
130    0
300    1
70     0
49     0
54     1
250    0
Name: heartdisease, dtype: int64

In [85]:
#models prediction of heart disease for the first 10 instances from the test set
model.predict(x_test[:10])

array([0, 0, 0, 1, 1, 1, 0, 0, 1, 1])

In [86]:
# score the accuracy of the model's predictions
accuracy = model.score(x_test,y_test)*100

In [88]:
#print the accuracy
print('the accuracy of the model is {0}%'.format(accuracy))

the accuracy of the model is 81.31868131868131%


## Section 2: AI in the fabric of society
### The privacy problem and k-anonymity
How do we fix the problem of de-anonymization is important for hospital database system to protect privacy. Sometimes the unique records in the database can easily find the personnel information even his or her personnel information such as name or address are not shown in the database. One example in this database:

In [19]:
np.unique(dataset.age)

array([29, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 74, 76, 77], dtype=int64)

In [20]:
unique_record = dataset.loc[dataset['age'] ==29]
unique_record

,age,sex,chestpain,restbps,cholesterol,fastingbs,restecg,thalach,exerciseang,oldpeak,slope,ca,thal,heartdisease
132,29,1,2,130,204,0,2,202,0,0.0,1,0,3,0


As you can see above, even though the detail personnel information is not shown, we can find a unique record who is 29 years old and is a male. Imagine if you know that your neighborhood called Able is 29 years old and is a male, and you know that he went to this hospital, then you can easily conclude that this guy has a large probability to be Able. With some other information in other datasets, the personnel information can be easily acquired, which is a horrible threat to people's privacy.

Read the ethical book and think about the problem and related solution.

Reading: *Ethical Algorithm*: Chapter 1, Algorithmic Privacy, A Bad Solution, (Page 26-31)